In [1]:
# IMPORT LIBRARIES
import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [2]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='TRANSFORMING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [3]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''--CUSTOMER INSIGHT CONTENT REPORT
            

with active_subs as (
                    SELECT
                    customer_id, SUM("PMA PRODUCTS") AS PMA_PRODUCTS, SUM("LMS PRODUCTS") AS LMS_PRODUCTS,
                    SUM("CONTENT PRODUCTS") AS CONTENT_PRODUCTS
                    FROM(
                    SELECT
                    customer_id,
                    CASE WHEN T1.item_id IN ('1_2','1_3','8_1') THEN SUM(T1.quantity) ELSE 0 END AS "PMA PRODUCTS",
                    CASE WHEN T1.item_id IN ('1_1','1_4','2_1','2_2','2_3','2_4','2_11','8_1','8_2','8_3','8_4') THEN SUM(T1.quantity) 
                    ELSE 0 END AS "LMS PRODUCTS",
                    CASE WHEN T1.item_id IN ('8_2','8_3') OR T2.item_family_id IN ('3') THEN SUM(quantity) ELSE 0 
                    END AS "CONTENT PRODUCTS"
                    FROM dbt_deploy.subscriptions T1
                    LEFT JOIN DBT_DEPLOY.STG_CHARGEBEE__ITEM T2 ON T1.ITEM_ID=T2.ITEM_ID
                    WHERE T1.status !='cancelled'
                    GROUP BY 1,T1.item_id,item_family_id
                    ORDER BY 1) AS SUB GROUP BY 1 ORDER BY 1     
                    ),

CONTENT_SUBS as (  SELECT
                        customer_id,
                        CONTENT_PROVIDER,
                        SUM(CONTENT_ARR_EUR) AS CONTENT_ARR_EUR
                FROM(
                SELECT
                T1.customer_id,
                T2.EXTERNAL_NAME AS CONTENT_PRODUCT_NAMES,
                CASE WHEN T2.EXTERNAL_NAME LIKE '%Skillsoft%' THEN 'Skillsoft'
                     WHEN T2.EXTERNAL_NAME LIKE '%Grape%' THEN 'Grape'
                     WHEN T2.EXTERNAL_NAME LIKE '%Opensesame%' THEN 'OpenSesame'
                     WHEN T2.EXTERNAL_NAME LIKE '%Go%' THEN 'Go'
                     WHEN T2.EXTERNAL_NAME LIKE '%Vado%' THEN 'Vado'
                     WHEN T2.EXTERNAL_NAME LIKE '%Skillcast%' THEN 'Skillcast'
                     WHEN T2.EXTERNAL_NAME LIKE '%Thrive%' THEN 'Thrive'
                     WHEN T2.EXTERNAL_NAME LIKE '%Awarego%' THEN 'Awarego'
                     WHEN T2.EXTERNAL_NAME LIKE '%Mybreev%' THEN 'Mybreev'
                     WHEN T2.EXTERNAL_NAME LIKE '%Eloomi Bundle%' THEN 'eloomi Bundle'
                     WHEN T2.EXTERNAL_NAME LIKE '%Kickstarter%' THEN 'eloomi Kickstarter'                     
                     WHEN T2.EXTERNAL_NAME LIKE '%Workplace%' THEN 'eloomi Respecatable Workplace'
                     WHEN T2.EXTERNAL_NAME LIKE '%Unlimited%' THEN 'eloomi Unlimited'
                     WHEN T2.EXTERNAL_NAME LIKE '%Infinite Awareness Essentials%' THEN 'infinite awareness essentials content'
                     WHEN T2.EXTERNAL_NAME LIKE '%Microlearn%' THEN 'Microlearn'
                     WHEN T2.EXTERNAL_NAME LIKE '%Mircolearn%' THEN 'Microlearn'
                     WHEN T2.EXTERNAL_NAME LIKE '%Skillshub%' THEN 'Skillshub'
                     WHEN T2.EXTERNAL_NAME LIKE '%Premium%' THEN 'Go'
                     WHEN T2.EXTERNAL_NAME LIKE '%25 Courses%' THEN 'Go'                     
                     WHEN T2.EXTERNAL_NAME LIKE '%Compliance Course library%' THEN 'eloomi'
                     WHEN T2.EXTERNAL_NAME LIKE '%CE Broker%' THEN 'CE Broker'
                     ELSE NULL END AS CONTENT_PROVIDER,
                SUM(T1.ARR_EUR) AS CONTENT_ARR_EUR
                from dbt_deploy.subscriptions T1
                LEFT JOIN DBT_DEPLOY.products T2 ON T1.ITEM_ID=T2.ITEM_ID
                where (T2.ITEM_FAMILY_ID IN ('3','4') or t1.item_id IN ('8_2','8_3'))
                and status !='cancelled'  GROUP BY 1,2 ORDER BY 1) AS SUB_TABLE GROUP BY 1,2 ),
               
ACTIVE_ARR as  ( SELECT 
                    CUSTOMER_ID, SUM(ARR_EUR) AS ACTIVE_ARR
                    FROM  dbt_deploy.subscriptions 
                    WHERE status != 'cancelled' AND start_date <= LAST_DAY(CURRENT_DATE()) AND end_date >=LAST_DAY(CURRENT_DATE())
                    GROUP BY 1 ),

contract_value as  ( SELECT 
                    CUSTOMER_ID, SUM(ARR_EUR) AS contract_Value
                    FROM  dbt_deploy.subscriptions 
                    WHERE status != 'cancelled' 
                    GROUP BY 1 ),
                    
max_license as (     SELECT
             CUSTOMER_ID,     GREATEST(
                COALESCE(active_content_licenses, 0),
                COALESCE(active_infinite_licenses, 0),
                COALESCE(active_people_licenses, 0),
                COALESCE(active_pm_licenses, 0),
                COALESCE(committed_people_licenses, 0),
                COALESCE(committed_infinite_licenses, 0),
                COALESCE(committed_pm_licenses, 0),
                COALESCE(committed_content_licenses, 0)
              ) AS greatest_license_volume
            From dbt_deploy.int_customer_licenses_calculated
                )


select 
        T1.economic_id,
        customer_name,
        account_status,
        ZEROIFNULL(ACTIVE_ARR) AS ACTIVE_ARR,
        T7.contract_Value,
        ZEROIFNULL(T8.ACTIVE_PEOPLE_LICENSES) as ACTIVE_PEOPLE_LICENSES ,
     ZEROIFNULL(T8.ACTIVE_PM_LICENSES) as ACTIVE_PM_LICENSES,
     ZEROIFNULL(T8.ACTIVE_INFINITE_LICENSES) as ACTIVE_INFINITE_LICENSES,
     ZEROIFNULL(T8.ACTIVE_CONTENT_LICENSES) as ACTIVE_CONTENT_LICENSES,
    ZEROIFNULL(T8.COMMITTED_PEOPLE_LICENSES) as COMMITTED_PEOPLE_LICENSES,
    ZEROIFNULL(T8.COMMITTED_PM_LICENSES) as COMMITTED_PM_LICENSES,
    ZEROIFNULL(T8.COMMITTED_INFINITE_LICENSES) as COMMITTED_INFINITE_LICENSES,
    ZEROIFNULL(T8.COMMITTED_CONTENT_LICENSES) as COMMITTED_CONTENT_LICENSES,
         CASE WHEN greatest_license_volume <= 49 THEN 'XS'
             WHEN greatest_license_volume <= 250  AND greatest_license_volume >= 50       THEN 'S'
             WHEN greatest_license_volume <= 1000 AND greatest_license_volume >= 251      THEN 'M'
             WHEN greatest_license_volume <= 5000  AND greatest_license_volume >= 1001     THEN 'L'
             WHEN greatest_license_volume <= 10000 AND greatest_license_volume >= 5001     THEN 'XL'
         ELSE 'XXL' END AS SIZE,
        t2.country_name as country,   
        t2.region,
        T6.fte_size,
CASE WHEN T6.fte_size < 250 THEN 'SMB' 
     WHEN (T6.fte_size > 249 AND 5001 > T6.fte_size)  THEN 'MM'
     WHEN T6.fte_size IS NULL THEN 'NULL'
     ELSE 'ENT' END AS Segment,
     T6.sub_industry,
     next_upcoming_renewal,
     T4.CONTENT_PROVIDER,
     T4.CONTENT_ARR_EUR
from dbt_deploy.customers t1
LEFT JOIN dbt_deploy.countries_and_regions t2 on t1.billing_address_country =t2.country_id
LEFT JOIN active_subs t3 on t3.customer_id =t1.customer_id
LEFT JOIN CONTENT_SUBS T4 ON T4.customer_id = t1.customer_id
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.INT_CUSTOMER_FIRMOGRAPHIC_ASSIGNED T6 ON T6.COMPANY_ID =T1.HUBSPOT_COMPANY_ID
LEFT JOIN ACTIVE_ARR T5 ON T5.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN contract_Value T7 ON T7.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN max_license T9 ON T9.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN dbt_deploy.int_customer_licenses_calculated T8 ON T8.CUSTOMER_ID = T1.CUSTOMER_ID
WHERE account_status !='cancelled' and T1.economic_id is not null 
            AND CONTENT_ARR_EUR IS NOT NULL
ORDER BY 4 ASC

''')


In [4]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [5]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [6]:
df_main.columns

Index(['ECONOMIC_ID', 'CUSTOMER_NAME', 'ACCOUNT_STATUS', 'ACTIVE_ARR',
       'CONTRACT_VALUE', 'ACTIVE_PEOPLE_LICENSES', 'ACTIVE_PM_LICENSES',
       'ACTIVE_INFINITE_LICENSES', 'ACTIVE_CONTENT_LICENSES',
       'COMMITTED_PEOPLE_LICENSES', 'COMMITTED_PM_LICENSES',
       'COMMITTED_INFINITE_LICENSES', 'COMMITTED_CONTENT_LICENSES', 'SIZE',
       'COUNTRY', 'REGION', 'FTE_SIZE', 'SEGMENT', 'SUB_INDUSTRY',
       'NEXT_UPCOMING_RENEWAL', 'CONTENT_PROVIDER', 'CONTENT_ARR_EUR'],
      dtype='object')

In [7]:
#CONVERT ALL THE COLUMN NAMES LOWER CASE

df_main.columns= df_main.columns.str.lower()


In [8]:
#Formats an integer with commas as thousand separators.

df_main.style.format({"CURRENT_MONTH_CONTRACT_VALUE": "{:,.0f}"})
df_main.style.format({"CONTENT_ARR_EUR": "{:,.0f}"})

,economic_id,customer_name,account_status,active_arr,contract_value,active_people_licenses,active_pm_licenses,active_infinite_licenses,active_content_licenses,committed_people_licenses,committed_pm_licenses,committed_infinite_licenses,committed_content_licenses,size,country,region,fte_size,segment,sub_industry,next_upcoming_renewal,content_provider,content_arr_eur
0,7347,CHANNEL 3 CONSULTING LTD,future,0.000000,22686.827200,0,0,0,0,0,0,100,100,S,United Kingdom,UK,100,SMB,PROFESSIONAL_TRAINING_COACHING,2027-10-31,Go,8854.760000
1,7336,Wagamama Limited,future,0.000000,133944.556400,0,0,0,0,0,0,9000,150,XL,United Kingdom,UK,9000,ENT,RESTAURANTS,2027-09-30,Go,6466.305000
2,5796,PikoHANA,active,2762.400000,2762.400000,0,0,50,50,0,0,0,0,S,Singapore,ROW,50,SMB,FINANCIAL_SERVICES,2024-10-31,OpenSesame,2117.840000
3,7335,EVA Airways Corporation,active,3222.800000,3222.800000,0,0,70,70,0,0,0,0,S,United States,US,10001,ENT,AIRLINES_AVIATION,2025-07-31,Skillsoft,1482.488000
4,7269,Temper Mechnical,active,3224.015456,3224.015456,0,0,50,50,0,0,0,0,S,United States,US,10,SMB,CONSTRUCTION,2025-03-31,Go,1731.104000
5,7323,XpresSpa,active,3277.587600,3277.587600,0,0,0,25,0,0,0,0,XS,United States,US,280,MM,CONSUMER_SERVICES,2027-06-02,infinite awareness essentials content,264.730000
6,5808,Zylpha Ltd.,non_renewing,3378.790000,3378.790000,0,0,25,25,0,0,0,0,XS,United Kingdom,UK,50,SMB,COMPUTER_SOFTWARE,2024-09-30,Go,2563.220000
7,7089,Sportsologygroup,active,3614.047920,3614.047920,0,0,25,25,0,0,0,0,XS,United States,US,25,SMB,ENTERTAINMENT,2025-06-30,Go,1136.037000
8,7084,Hightowers Petroleum,active,3685.594080,3685.594080,0,0,25,25,0,0,0,0,XS,United States,US,50,SMB,OIL_ENERGY,2025-06-30,Go,1160.208000
9,7101,Jorissen & De Jonck,active,3999.000000,3999.000000,0,0,25,10,0,0,0,0,XS,Belgium,NORBEN,25,SMB,STAFFING_AND_RECRUITING,2026-12-31,Go,420.000000


In [9]:
df_main.head(2)

,economic_id,customer_name,account_status,active_arr,contract_value,active_people_licenses,active_pm_licenses,active_infinite_licenses,active_content_licenses,committed_people_licenses,...,committed_content_licenses,size,country,region,fte_size,segment,sub_industry,next_upcoming_renewal,content_provider,content_arr_eur
0,7347,CHANNEL 3 CONSULTING LTD,future,0.0,22686.8272,0,0,0,0,0,...,100,S,United Kingdom,UK,100,SMB,PROFESSIONAL_TRAINING_COACHING,2027-10-31,Go,8854.760
1,7336,Wagamama Limited,future,0.0,133944.5564,0,0,0,0,0,...,150,XL,United Kingdom,UK,9000,ENT,RESTAURANTS,2027-09-30,Go,6466.305


In [10]:
# SET UP DATE FOR NAMING THE OUTPUT FILES
from dateutil.relativedelta import relativedelta
period_end_date = date.today() - relativedelta(months=1)
report_date = period_end_date.strftime('%b-%y')
report_date

'Aug-24'

In [11]:
excelfilename = report_date +"-Customer Content Insight Report.xlsx"

In [12]:
# OUTPUT FINAL FILE WITH ALL DIFFERENCES
df_main.to_excel(excelfilename, sheet_name=report_date, index=False)
